In [ ]:
import pandas as pd

listings = pd.read_csv('./data/listings.csv')
reviews = pd.read_csv('./data/reviews.csv')

In [ ]:
reviews.columns

In [ ]:
listings.columns

## Reviewer

In [ ]:
reviewer = pd.DataFrame()
reviewer['reviewer_id'] = reviews['reviewer_id']
reviewer['reviewer_name'] = reviews['reviewer_name']

In [3]:
reviewer_final = reviewer.drop_duplicates(subset=['reviewer_id'])
reviewer_final = reviewer_final.reset_index(drop=True)

In [ ]:
reviewer_final.to_csv('./data_sql/reviewer.csv', index=False, sep=';')

## Date

In [ ]:
from datetime import datetime

def get_day(date): 
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.day

def get_month(date):
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.month

def get_year(date):
    data = datetime.strptime(date,'%Y-%m-%d')
    return data.year

In [38]:
date = pd.DataFrame() 
date['date'] = reviews['date']

In [42]:
#drop duplicates
date = date.drop_duplicates()
date = date.reset_index(drop=True)

In [ ]:
#foreign key for day

size_date = len(date.index)
date['day_id'] = [i for i in range(size_date)]

for i in date['date'].index:
    date['day_id'][i] = get_day(date['date'][i])

In [ ]:
#foreign key for month

size_date = len(date.index)
date['month_id'] = [i for i in range(size_date)]
for i in date['date'].index:
    date['month_id'][i] = get_month(date['date'][i])

In [52]:
# Table day
day = pd.DataFrame()
day['day_number'] = [i for i in range(1,32)]

In [ ]:
# Table month
month = pd.DataFrame()
month['month_number'] = [i for i in range(1,13)]
month['month_name'] = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [54]:
max = date['date'].max()
min = date['date'].min()

print(max, min)

2023-09-11 2010-07-22


In [31]:
# Table year -> min_year(date) - max_year(date)
year = pd.DataFrame()
year['year_number'] = [i for i in range(2010,2024)]

In [ ]:
# Foreign key of year_id

size_date = len(date.index)
date['year_id'] = [i for i in range(size_date)]
for i in date['date'].index:
    year_number = get_year(date['date'][i])
    row_num = year[year['year_number']==year_number].index
    date['year_id'][i] = row_num[0]+1

#### Date - reading the csv

In [9]:
date_csv = pd.read_csv('./data_sql/date.csv', sep=';')

In [12]:
date_csv['day_number'] = date_csv['day_id'].copy()
date_csv['month_number'] = date_csv['month_id'].copy()

In [23]:
def get_month_name(month_id):
    month_row = month[month['month_number']==month_id]
    return month_row['month_name'].iloc[0]

In [29]:
size_date = len(date_csv.index)
date_csv['month_name'] = [i for i in range(size_date)]

for i in date_csv['month_id'].index:
    month_name = get_month_name(date_csv['month_id'][i])
    date_csv['month_name'][i] = month_name

C:\Users\filip\AppData\Local\Temp\ipykernel_3684\4036878054.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_csv['month_name'][i] = month_name


In [34]:
def get_year_number(year_id):
    return 2010 + year_id -1

In [39]:
size_date = len(date_csv.index)
date_csv['year_number'] = [i for i in range(size_date)]

for i in date_csv['year_id'].index:
    year_number = get_year_number(date_csv['year_id'][i])
    date_csv['year_number'][i] = year_number

C:\Users\filip\AppData\Local\Temp\ipykernel_3684\1921800919.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_csv['year_number'][i] = year_number


In [41]:
date_csv.to_csv('./data_sql/date.csv', sep=";", index=False)

## Location

In [140]:
location = pd.DataFrame()
location['latitude'] = listings['latitude']
location['longitude'] = listings['longitude']

In [141]:
location = location.drop_duplicates().reset_index(drop=True)

In [142]:
state = pd.DataFrame({'state_id':1, 'state':['Porto'], 'country_id': 1})

In [143]:
country = pd.DataFrame({'country_id': 1, 'country': ['Portugal'], 'country_code': ['PT']})

# Create a new dataframe that came from listings that take the city

city = pd.DataFrame()


city["city"] = listings["neighbourhood_group_cleansed"].drop_duplicates().reset_index(drop=True)
size_city = len(city.index)
city["city_id"] = [i for i in range(1, size_city+1)]
city["country_id"] = 1

# Create a new dataframe that came from listings that take the neighbourhood

neighbourhood = pd.DataFrame()

# Extract the 'neighbourhood_cleansed' column and drop duplicates
neighbourhood['combined_neighbourhood'] = listings['neighbourhood_cleansed'].drop_duplicates().reset_index(drop=True)

# Assuming 'city_column' is the column in 'listings' containing city information
neighbourhood = pd.merge(neighbourhood, listings[['neighbourhood_cleansed', 'neighbourhood_group_cleansed']], 
                         left_on='combined_neighbourhood', right_on='neighbourhood_cleansed', how='left')

neighbourhood.rename(columns={'neighbourhood_group_cleansed': 'city_column'}, inplace=True)
neighbourhood.drop(['neighbourhood_cleansed'], axis=1, inplace=True)
neighbourhood.drop_duplicates().reset_index(drop=True, inplace=True)


# Split the combined neighborhood string into individual neighborhoods using both commas and " e "
neighbourhood['neighbourhood'] = neighbourhood['combined_neighbourhood'].str.split(',| e ')

# Explode the DataFrame, creating a new row for each neighborhood
neighbourhood = neighbourhood.explode('neighbourhood')

# Drop unnecessary columns and rename columns
neighbourhood.rename(columns={'city_column': 'city'}, inplace=True)
neighbourhood.drop_duplicates().reset_index(drop=True, inplace=True)
neighbourhood = pd.merge(neighbourhood, city, on="city", how='left')
neighbourhood.drop(['combined_neighbourhood', 'city', 'country_id'], axis=1, inplace=True)

In [144]:
city.rename(columns={'country_id':'state_id'}, inplace=True)

In [145]:
city.drop(['city_id'], axis=1, inplace=True)

In [146]:
neighbourhood = neighbourhood.drop_duplicates().reset_index(drop=True)

In [147]:
location['neighbourhood_cleansed'] = listings['neighbourhood_cleansed']
location['neighbourhood_id'] = 0

location['neighbourhood_cleansed'] = location['neighbourhood_cleansed'].str.split(',| e ')

index = 0
for listn in location['neighbourhood_cleansed']:
    for n in listn:
        location.loc[index, 'neighbourhood_id'] = neighbourhood[neighbourhood['neighbourhood']==n].index[0] + 1

    index += 1

location = location.drop(['neighbourhood_cleansed'], axis=1)

In [149]:
location.to_csv('./data_sql/location.csv', sep=";", index=False)

In [151]:
neighbourhood.to_csv('./data_sql/neighbourhood.csv', index=False, sep=";")

In [153]:
city.to_csv('./data_sql/city.csv', sep=";", index=False)

In [156]:
state.drop(['state_id'], axis=1, inplace=True)

In [158]:
state.to_csv('./data_sql/state.csv', sep=";", index=False)

In [160]:
country.drop(['country_id'], axis=1, inplace=True)

In [162]:
country.to_csv('./data_sql/country.csv', sep=";", index=False)

## Host

In [12]:
# selected_columns = ['host_response_time', 'host_response_rate', 'host_acceptance_rate']
selected_columns = ['host_id', 'host_name', 'host_url', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'host_is_superhost', 'host_about',
                    'host_response_time', 'host_response_rate', 'host_acceptance_rate', 
                    'host_verifications']

host = listings[selected_columns].copy()
host.rename(columns={'host_about':'description'}, inplace=True)

In [13]:
selected_columns = ['host_response_time', 'host_response_rate', 'host_acceptance_rate']

host_response = listings[selected_columns].copy()

In [14]:
host_response = host_response.drop_duplicates().reset_index(drop=True)

In [20]:
host_response = host_response.fillna(-1)

In [22]:
def get_host_response_id(host_id):
    listing_host = listings[listings['host_id']==host_id]
    list_host_response_time = listing_host['host_response_time'].iloc[0]
    list_host_response_rate = listing_host['host_response_rate'].iloc[0]
    list_host_acceptance_rate = listing_host['host_acceptance_rate'].iloc[0]

    # Check if values are NaN
    is_response_time_nan = pd.isna(list_host_response_time)
    is_response_rate_nan = pd.isna(list_host_response_rate)
    is_acceptance_rate_nan = pd.isna(list_host_acceptance_rate)

    if is_response_time_nan:
        list_host_response_time = -1
    if is_response_rate_nan:
        list_host_response_rate = -1
    if is_acceptance_rate_nan:
        list_host_acceptance_rate = -1

    row = host_response[(host_response['host_response_time']==list_host_response_time)
                         & (host_response['host_response_rate']==list_host_response_rate)
                         & (host_response['host_acceptance_rate']==list_host_acceptance_rate)]
    
    return row.index[0] + 1 

In [25]:
# host_response_id
size_host = len(host.index)
host['host_response_id'] = [i for i in range(size_host)]

for i in host['host_id'].index:
    host_id = host['host_id'][i]
    host['host_response_id'][i] = get_host_response_id(host_id)

C:\Users\filip\AppData\Local\Temp\ipykernel_4768\692988943.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host['host_response_id'][i] = get_host_response_id(host_id)


In [29]:
host_verification = pd.DataFrame()
host_verification['host_verifications'] = listings['host_verifications']

In [30]:
host_verification = host_verification.drop_duplicates().reset_index(drop=True)

In [32]:
def get_host_verifications_id(host_id):
    listing_host = listings[listings['host_id']==host_id]
    list_host_verifications = listing_host['host_verifications'].iloc[0]
    
    row = host_verification[host_verification['host_verifications']==list_host_verifications]
    return row.index[0] + 1 

In [33]:
# host_verifications_id -> run
size_host = len(host.index)
host['host_verifications_id'] = [i for i in range(size_host)]

for i in range(size_host):
    host_id = host['host_id'][i]
    host['host_verifications_id'][i] = get_host_verifications_id(host_id)

C:\Users\filip\AppData\Local\Temp\ipykernel_4768\482111441.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host['host_verifications_id'][i] = get_host_verifications_id(host_id)


In [37]:
host.columns

Index(['host_id', 'host_name', 'host_url', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'host_is_superhost',
       'description', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_verifications', 'host_response_id',
       'host_verifications_id'],
      dtype='object')

In [38]:
host = host.drop_duplicates().reset_index(drop=True)

In [39]:
host.to_csv('./data_sql/host.csv', sep=";", index=False)

## Property

In [121]:
selected_columns = ['accommodates', 'bedrooms', 'beds', 'amenities', 'room_type', 'property_type', 'bathrooms_text']

property = listings[selected_columns].copy()
property.rename(columns={'bathrooms_text':'bathrooms'}, inplace=True)

In [122]:
property = property.drop_duplicates().reset_index(drop=True)

In [42]:
property.columns

Index(['accommodates', 'bedrooms', 'beds', 'amenities', 'room_type',
       'property_type', 'bathrooms', 'property_id'],
      dtype='object')

In [5]:
property.to_csv('./data_sql/property.csv', sep=";", index=False)

# Review

In [6]:
review = pd.DataFrame()
review['reviewer_id'] = reviews['reviewer_id']
review['review'] = reviews['comments']


In [7]:
def get_date_id(date_str):
    return date[date['date']==date_str].index[0]+1

In [10]:
date = pd.read_csv('./data_sql/date.csv', sep=';')

In [11]:
# data_id
size_review = len(review.index)
review['date_id'] = [i for i in range(size_review)]

for i in range(size_review):
    review['date_id'][i] = get_date_id(reviews['date'][i])

C:\Users\filip\AppData\Local\Temp\ipykernel_14204\3970891828.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['date_id'][i] = get_date_id(reviews['date'][i])


In [16]:
review['listing_id'] = reviews['listing_id']

In [31]:
def get_host_id(listings_id):
    l = listings[listings['id']==listings_id]
    return l['host_id'].tolist()[0]

In [37]:
# host_id 

size_review = len(review.index)
review['host_id'] = [i for i in range(size_review)]

for i in range(size_review):
    review['host_id'][i] = get_host_id(review['listing_id'][i])

C:\Users\filip\AppData\Local\Temp\ipykernel_14204\2635959504.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['host_id'][i] = get_host_id(review['listing_id'][i])


In [92]:
property_id = property.copy()

In [93]:
property_id = property_id.fillna(-1)

In [123]:
def get_property_id(listing_id):
    l = listings[listings['id']==listing_id]

    accommodates = l['accommodates'].iloc[0]
    bedrooms = l['bedrooms'].iloc[0]
    beds = l['beds'].iloc[0]
    amenities = l['amenities'].iloc[0]
    room_type = l['room_type'].iloc[0]
    property_type = l['property_type'].iloc[0]
    bathrooms = l['bathrooms_text'].iloc[0]

    is_bedrooms_nan = pd.isna(bedrooms)
    is_beds_nan = pd.isna(beds)
    is_bathrooms_nan = pd.isna(bathrooms)


    if is_bedrooms_nan:
        bedrooms = -1
    if is_beds_nan:
        beds = -1
    if is_bathrooms_nan:
        bathrooms = -1
    
    row = property_id[(property_id['accommodates']==accommodates)
                      & (property_id['bedrooms'] == bedrooms)
                      & (property_id['beds'] == beds)
                      & (property_id['amenities'] == amenities)
                      & (property_id['room_type'] == room_type)
                      & (property_id['property_type'] == property_type)
                      & (property_id['bathrooms'] == bathrooms)]
    
    return row.index[0] + 1

In [125]:
# property_id

size_review = len(review.index)
review['property_id'] = [i for i in range(size_review)]

for i in range(size_review):
    review['property_id'][i] = get_property_id(review['listing_id'][i])


C:\Users\filip\AppData\Local\Temp\ipykernel_14204\22328400.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['property_id'][i] = get_property_id(review['listing_id'][i])


In [128]:
review.to_csv('./data_sql/review.csv', sep=";", index=False)

In [129]:
review.columns

Index(['reviewer_id', 'review', 'date_id', 'listing_id', 'host_id',
       'property_id'],
      dtype='object')

# Listing

In [163]:
#listing -> listing_id = id, host_id = host_id, price -> price

# selected_columns = ['availability_30', 'availability_60', 'availability_90', 'availability_365']
# selected_columns = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

selected_columns = ['id', 'host_id', 'price', 'description', 
                    'availability_30', 'availability_60', 'availability_90', 'availability_365', 
                    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']
listing = listings[selected_columns].copy()
listing.rename(columns={'id':'listing_id'}, inplace=True)

In [164]:
listing.columns

Index(['listing_id', 'host_id', 'price', 'description', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month'],
      dtype='object')

In [215]:
# property_id

size_listing = len(listing.index)
listing['property_id'] = [i for i in range(size_listing)]

for i in range(size_listing):
    listing['property_id'][i] = get_property_id(listing['listing_id'][i])


C:\Users\filip\AppData\Local\Temp\ipykernel_14204\1556357291.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing['property_id'][i] = get_property_id(listing['listing_id'][i])


In [214]:
def get_location_id(listing_id):
    l = listings[listings['id']==listing_id]

    if l.empty:
        return pd.NA
    
    latitude = l['latitude'].iloc[0]
    longitude = l['longitude'].iloc[0]

    row = location[(location['latitude'] == latitude)
                   & (location['longitude'] == longitude)]
    
    if not row.empty:
        return row.index[0] + 1
    else :
        return pd.NA

In [195]:
# location_id
size_listing = len(listing.index)
listing['location_id'] = [i for i in range(size_listing)]

for i in range(size_listing):
    listing['location_id'][i] = get_location_id(listing['listing_id'][i])


C:\Users\filip\AppData\Local\Temp\ipykernel_14204\52127130.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing['location_id'][i] = get_location_id(listing['listing_id'][i])


In [217]:
listing.to_csv('./data_sql/listing.csv', sep=";", index=False)